# How to Measurements

## Overview
This notebook shows how to use the `measurements` of the papyrus package. 
It will start by showing how to create a `Measurement` object and how to connect it 
to a recorder. 

In [6]:
import papyrus.measurements as m
import numpy as np

All available measurements are listed in the `papyrus.measurements` module.

Of these, the `BasedMeasurement` is the parent class required to create a new measurement.
All other measurements are based on this class and can be used for recording data.

In [11]:
# All available measurements

m.__all__

['BaseMeasurement',
 'NTKTrace',
 'NTKEntropy',
 'NTKSelfEntropy',
 'NTKEigenvalues',
 'NTKMagnitudeDistribution',
 'Loss',
 'Accuracy',
 'NTK']

In [13]:
# Define a measurement for a loss function

def loss_fn(predictions, targets):
    return np.mean((predictions - targets) ** 2, keepdims=True)

loss = m.Loss(
    name='loss', # Name of the measurement
    apply_fn=loss_fn # The function that will be called to compute the loss
)
print(f"Neural state keys: {loss.neural_state_keys}")

# Defining the neural state
neural_state = {
    'predictions': np.array([[0.5], [0.6]]),
    'targets': np.array([[0.], [0.]])
}

print(f"The loss is {loss( **neural_state )}")

Neural state keys: ['predictions', 'targets']
The loss is [[0.25]
 [0.36]]


As you can see, calling a `Measurement` object will execute the measurement and return the result.

### Defining Neural States

The concept behind neural states is to define the state of a neural network at a given time.
This could theoretically take any form and the recorders allow for the definition of custom states.
However, the default state can take the following form:
```python
neural_state = {
    "loss": np.ndarray, 
    "accuracy": np.ndarray,
    "predictions": np.ndarray,
    "targets": np.ndarray,
    "ntk": np.ndarray,
}
```
Note that this defines one neural state.
The keys of the dictionary have to match the keys of the `Measurement` objects. 

Each value of a neural state has to be a numpy array, with its first dimension defining 
the number of sub-states, the entire neural state is composed of.
For all keys, the number of sub-states has to be the same.

## Default Measurements

### Loss and Accuracy

In [14]:
#############################################
### Defining a loss function as a measurement
#############################################

def loss_fn(predictions, targets):
    return np.mean((predictions - targets) ** 2, keepdims=True)

loss = m.Loss(
    name='loss', # Name of the measurement
    apply_fn=loss_fn # The function that will be called to compute the loss
)

# Defining the neural state
neural_state = {
    'predictions': np.array([[0.5], [0.6]]),
    'targets': np.array([[0.], [0.]])
}

print(f"The loss is {loss( **neural_state )}")


#############################################
### Measuring pre-computed loss values 
#############################################

loss = m.Loss(name='loss')

# Defining the neural states with a pre-computed loss
neural_state = { 'loss': np.array([ [0.5], [0.6] ]) }

print(f"The loss is {loss(**neural_state)}")

The loss is [[0.25]
 [0.36]]
The loss is [[0.5]
 [0.6]]


In [16]:
#############################################
### Defining an accuracy measurement
#############################################

def accuracy_fn(predictions, targets):
    return np.sum(np.argmax(predictions, axis=1) == np.argmax(targets, axis=1)) / len(predictions)

neural_state = {
    'predictions': np.array([ [[0.8, 0.2], [0.6, 0.4], [0.3, 0.7]] ]),
    'targets': np.array([ [[1, 0], [0, 1], [0, 1]] ])
}

accuracy = m.Accuracy(
    name='accuracy',
    apply_fn=accuracy_fn
)

print(f"The accuracy is {accuracy(**neural_state)}")


#############################################
### Measuring pre-computed accuracy values
#############################################

accuracy = m.Accuracy(name='accuracy')

neural_state = { 'accuracy': np.array([ [0.8, 0.6, 0.3] , [0.2, 0.4, 0.7] ])}

print(f"The accuracy is {accuracy(**neural_state)}")

The accuracy is [0.66666667]
The accuracy is [[0.8 0.6 0.3]
 [0.2 0.4 0.7]]


### NTK Properties

In [17]:
import inspect

a = inspect.signature(accuracy_fn).parameters
a.keys()

odict_keys(['predictions', 'targets'])